# Example block-wise adaptation

In [16]:
import numpy as np
from numpy import unravel_index
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import ot
import scipy.io
import mne          
from mne.decoding import CSP
mne.set_log_level(verbose='warning') #to avoid info at terminal
import matplotlib.pyplot as pl
from random import seed
seed(30)
from MIOTDAfunctions import*

# get the functions from RPA package
import rpa.transfer_learning as TL

from pyriemann.classification import MDM
from pyriemann.estimation import Covariances
from pyriemann.utils.base import invsqrtm
import timeit

#ignore warning 
from warnings import simplefilter

# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)

In [2]:
results_acc=[]
results_all=[]
results_all_inv=[]

rango_cl = [0.1, 0.5, 1, 2, 5, 10, 20]
rango_e = [0.1, 0.5, 1, 2, 5, 10, 20]
metric = 'sqeuclidean'
outerkfold = 10 # for faster online computation select a lower value
innerkfold = dict(nfold=10, train_size=0.8)

## Methods definition

In [3]:
def SC(Gte, Yte, lda):
    
    start = timeit.default_timer()
    
    acc = lda.score(Gte, Yte)
    
    stop = timeit.default_timer()
    time = stop - start
    
    return acc, time 

In [4]:
def SR(Data_S2, Labels_S2, re, Xtr, Ytr, Xte, Yte):
    
    start = timeit.default_timer()
    
    #Get Data
    Xtr2add = Data_S2[0:20*re+20]
    Ytr2add = Labels_S2[0:20*re+20]
    
    Xtr2 = np.vstack(((Xtr, Xtr2add)))
    Ytr2 = np.hstack(((Ytr, Ytr2add)))
        
    Ytr2 = Ytr2[len(Ytr2add):]
    Xtr2 = Xtr2[len(Ytr2add):]

    # Create a new CSP
    csp = CSP(n_components=6, reg='empirical', log=True, norm_trace=False, cov_est='epoch')
    
    #learn new csp filters
    Gtr = csp.fit_transform(Xtr2,Ytr2)
    
    #learn new lda
    lda = LinearDiscriminantAnalysis()
    lda.fit(Gtr, Ytr2)

    # Apply on new test data
    Gte = csp.transform(Xte)
    #ldatest
    acc = lda.score(Gte, Yte)
    
    # time
    stop = timeit.default_timer()
    time = stop - start
    
    return acc, time 

In [5]:
def Sinkhorn_Transport(Gtr, Ytr, Gval, Yval, Gte, Yte, rango_e, metric, outerkfold, innerkfold, M):
    
    lda = LinearDiscriminantAnalysis()
    
    # Subset selection
    G_FOTDAs_, Y_FOTDAs_, regu_FOTDAs_=\
    SelectSubsetTraining_OTDAs(Gtr, Ytr, Gval, Yval, rango_e, lda, metric, outerkfold, innerkfold, M)

    #time
    start = timeit.default_timer()
    
    Gtr_daot = G_FOTDAs_
    Ytr_daot = Y_FOTDAs_ 
    
    otda = ot.da.SinkhornTransport(metric=metric, reg_e=regu_FOTDAs_)
    #learn the map
    otda.fit(Xs=Gtr_daot, ys=Ytr_daot, Xt=Gval)
    
    #apply the mapping over source data
    transp_Xs = otda.transform(Xs=Gtr)

    # train a new classifier bases upon the transform source data
    lda.fit(transp_Xs, Ytr)
    
    # Compute acc
    yt_predict = lda.predict(Gte)
    acc = accuracy_score(Yte, yt_predict)
    
    # time
    stop = timeit.default_timer()
    time = stop - start  
    
    return acc, time

In [6]:
def GroupLasso_Transport(Gtr, Ytr, Gval, Yval, Gte, Yte, rango_e, rango_cl, metric, outerkfold, innerkfold, M):
    
    lda = LinearDiscriminantAnalysis()
    
    # Subset selection
    G_FOTDAl1l2_, Y_FOTDAl1l2_, regu_FOTDAl1l2_=\
    SelectSubsetTraining_OTDAl1l2(Gtr, Ytr, Gval, Yval, rango_e, rango_cl, lda, metric, outerkfold, innerkfold, M)
    
    #time
    start = timeit.default_timer()
    
    Gtr_daot = G_FOTDAl1l2_
    Ytr_daot = Y_FOTDAl1l2_
    
    otda = ot.da.SinkhornL1l2Transport(metric = metric ,reg_e = regu_FOTDAl1l2_[0], reg_cl = regu_FOTDAl1l2_[1])
    otda.fit(Xs=Gtr_daot, ys=Ytr_daot, Xt=Gval)

    #transport taget samples onto source samples
    transp_Xs = otda.transform(Xs=Gtr)

    # train a new classifier bases upon the transform source data
    lda.fit(transp_Xs,Ytr)

    # Compute acc
    yt_predict = lda.predict(Gte)
    acc = accuracy_score(Yte, yt_predict)
    
    # time
    stop = timeit.default_timer()
    time = stop - start 
        
    
    return acc, time

In [7]:
def Backward_Sinkhorn_Transport(Gtr, Ytr, Gval, Yval, Gte, Yte, rango_e, lda, metric, outerkfold, innerkfold, M):
    # the classifier already trained is an input of the function

    # Subset selection
    G_BOTDAs_, Y_BOTDAs_, regu_BOTDAs_=\
    SelectSubsetTraining_BOTDAs(Gtr, Ytr, Gval, Yval, rango_e, lda, metric, outerkfold, innerkfold, M)
    
    # time
    start = timeit.default_timer()
    
    Gtr_botda = G_BOTDAs_
    Ytr_botda = Y_BOTDAs_
    
    # Transport plan
    botda = ot.da.SinkhornTransport(metric=metric, reg_e=regu_BOTDAs_)
    botda.fit(Xs=Gval, ys=Yval, Xt=Gtr_botda)
    
    #transport testing samples
    transp_Xt_backward = botda.transform(Xs=Gte)
    
    # Compute accuracy without retraining    
    yt_predict = lda.predict(transp_Xt_backward)
    acc = accuracy_score(Yte, yt_predict)
    
    # time
    stop = timeit.default_timer()
    time = stop - start
    
    return acc, time

In [8]:
def Backward_GroupLasso_Transport(Gtr, Ytr, Gval, Yval, Gte, Yte, rango_e, rango_cl, lda, metric, outerkfold, innerkfold, M):
    # the classifier already trained is an input of the function
    
    # Subset selection 
    G_BOTDAl1l2_, Y_BOTDAl1l2_, regu_BOTDAl1l2_=\
    SelectSubsetTraining_BOTDAl1l2(Gtr, Ytr, Gval, Yval, rango_e, rango_cl, lda, metric, outerkfold, innerkfold, M)
    
    #time
    start = timeit.default_timer()
    Gtr_botda = G_BOTDAl1l2_
    Ytr_botda = Y_BOTDAl1l2_
    
    botda = ot.da.SinkhornL1l2Transport(metric=metric, reg_e=regu_BOTDAl1l2_[0], reg_cl=regu_BOTDAl1l2_[1])
    botda.fit(Xs=Gval, ys=Yval, Xt=Gtr_botda)
    
    #transport testing samples
    transp_Xt_backward=botda.transform(Xs=Gte)
    
    # Compute accuracy without retraining    
    yt_predict = lda.predict(transp_Xt_backward)
    acc = accuracy_score(Yte, yt_predict)
    
    # time
    stop = timeit.default_timer()
    time = stop - start
    
    
    return acc, time

In [9]:
def RPA(Xtr,Xval,Xte,Ytr,Yval,Yte):
    
    # time
    start = timeit.default_timer()
    # cov matrix estimation
    cov_tr = Covariances().transform(Xtr)
    cov_val= Covariances().transform(Xval)
    cov_te = Covariances().transform(Xte)
        
    clf = MDM()
    source={'covs':cov_tr, 'labels': Ytr}
    target_org_train={'covs':cov_val, 'labels': Yval}
    target_org_test={'covs':cov_te, 'labels': Yte}
    
    # re-centered matrices
    source_rct, target_rct_train, target_rct_test = TL.RPA_recenter(source, target_org_train, target_org_test)   
    # stretched the re-centered matrices
    source_rcs, target_rcs_train, target_rcs_test = TL.RPA_stretch(source_rct, target_rct_train, target_rct_test)
    # rotate the re-centered-stretched matrices using information from classes
    source_rpa, target_rpa_train, target_rpa_test = TL.RPA_rotate(source_rcs, target_rcs_train, target_rcs_test)
    
    # get data
    covs_source, y_source = source_rpa['covs'], source_rpa['labels']
    covs_target_train, y_target_train = target_rpa_train['covs'], target_rpa_train['labels']
    covs_target_test, y_target_test = target_rpa_test['covs'], target_rpa_test['labels']
    
    # append train and validation data
    covs_train = np.concatenate([covs_source, covs_target_train])
    y_train = np.concatenate([y_source, y_target_train])
    
    # train
    clf.fit(covs_train, y_train)
    
    # test
    covs_test = covs_target_test
    y_test = y_target_test
    y_pred = clf.predict(covs_test)
    
    #acc
    acc = accuracy_score(Yte, y_pred)
    
    # time
    stop = timeit.default_timer()
    time = stop - start
    
    return acc, time

In [10]:
def EU(Xtr,Xval,Xte,Ytr,Yval,Yte):
    
    # time
    start = timeit.default_timer()
    # Estimate single trial covariance
    cov_tr = Covariances().transform(Xtr)
    cov_val= Covariances().transform(Xval)
    
    Ctr = cov_tr.mean(0)
    Cval = cov_val.mean(0)
    
    # aligment
    Xtr_eu = np.asarray([np.dot(invsqrtm(Ctr), epoch) for epoch in Xtr])
    Xval_eu = np.asarray([np.dot(invsqrtm(Cval), epoch) for epoch in Xval])
    Xte_eu = np.asarray([np.dot(invsqrtm(Cval), epoch) for epoch in Xte])

    # append train and validation data
    x_train = np.concatenate([Xtr_eu, Xval_eu])
    y_train = np.concatenate([Ytr, Yval])

    # train new csp+lda
    csp = CSP(n_components=6, reg='empirical', log=True, norm_trace=False, cov_est='epoch')
    # learn csp filters
    Gtr = csp.fit_transform(x_train,y_train)
    
    # learn lda
    lda = LinearDiscriminantAnalysis()
    lda.fit(Gtr,y_train)
    
    # test
    Gte = csp.transform(Xte_eu)  
    # acc
    acc = lda.score(Gte, Yte)
    # time
    stop = timeit.default_timer()
    time = stop - start
        
    return acc, time

## Load and filter data 

In [11]:
fName = 'Data/DataSession1_S9.mat'
s = scipy.io.loadmat(fName)

Data_S1=s["X"]
Labels_S1=s["y"]
Labels_S1=np.squeeze(Labels_S1)

#filterting with mne
[nt, nc, ns]=np.shape(Data_S1)
Data_S1=np.reshape(Data_S1, [nt, nc*ns])
Data_S1=mne.filter.filter_data(Data_S1, 128, 8, 30)
Data_S1=np.reshape(Data_S1, [nt,nc,ns])

fName = 'Data/DataSession2_S9.mat'
s2 = scipy.io.loadmat(fName)

Data_S2=s2["X"]
Labels_S2=s2["y"]
Labels_S2=np.squeeze(Labels_S2)

#filterting with mne
[nt, nc, ns]=np.shape(Data_S2)
Data_S2=np.reshape(Data_S2, [nt, nc*ns])
Data_S2=mne.filter.filter_data(Data_S2, 128, 8, 30)
Data_S2=np.reshape(Data_S2, [nt,nc,ns])

### Learn CSP+LDA from source data (Data_S1)

In [12]:
Xtr = Data_S1
Ytr = Labels_S1
csp = CSP(n_components=6, reg='empirical', log=True, norm_trace=False, cov_est='epoch')
#learn csp filters
Gtr = csp.fit_transform(Xtr, Ytr)
#learn lda
lda = LinearDiscriminantAnalysis()
lda.fit(Gtr,Ytr)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

### For each run of 20 trials each, make the data adaptation

In [13]:
for re in range(0,7):
    print('Running testing RUN={:1.0f}'.format(re))
    #testing run
    Xte = Data_S2[0+20*(re+1):20*(re+1)+20]
    Yte = Labels_S2[0+20*(re+1):20*(re+1)+20]
    #transportation set-prior data
    Xval = Data_S2[0:20*re+20]
    Yval = Labels_S2[0:20*re+20]
    
    #feature computation
    Gval = csp.transform(Xval)
    Gte = csp.transform(Xte)
    
    M = len(Yval) #for the source subset selection
    
    # SC  
    acc_sc, time_sc = SC(Gte, Yte, lda)
    
    # SR
    acc_sr, time_sr = SR(Data_S2, Labels_S2, re, Xtr, Ytr, Xte, Yte)
    
    #%% # Sinkhorn Transport
    acc_fotdas, time_fs = Sinkhorn_Transport(Gtr, Ytr, Gval, Yval, Gte, Yte, rango_e, metric, outerkfold, innerkfold, M)
    
    #%% # Group-Lasso Transport
    acc_fotdal1l2, time_fg = GroupLasso_Transport(Gtr, Ytr, Gval, Yval, Gte, Yte, rango_e, rango_cl, metric, outerkfold, innerkfold, M)
    
    #%% # Backward Sinkhorn Transport
    acc_botdas, time_bs = Backward_Sinkhorn_Transport(Gtr, Ytr, Gval, Yval, Gte, Yte, rango_e, lda, metric, outerkfold, innerkfold, M)
    
    #%% # Backward Group-Lasso Transport
    acc_botdal1l2, time_bg = Backward_GroupLasso_Transport(Gtr, Ytr, Gval, Yval, Gte, Yte, rango_e, rango_cl, lda, metric, outerkfold, innerkfold, M)
    
    # Riemann
    acc_rpa, time_rpa = RPA(Xtr, Xval, Xte, Ytr, Yval, Yte)
    
    # Euclidean
    acc_eu, time_eu = EU(Xtr, Xval, Xte, Ytr, Yval, Yte)
    
    # print results
    # accuracy
    acc = {}
    acc["sc"] = acc_sc
    acc["sr"] = acc_sr
    # acc["rpa"] = acc_rpa
    acc["ea"] = acc_eu
    acc["fotda_s"] = acc_fotdas
    acc["fotda_l1l2"] = acc_fotdal1l2
    acc["botda_s"] = acc_botdas
    acc["botda_l1l2"] = acc_botdal1l2
    
    # computing time
    time = {}
    time["sr"] = round(time_sr,3)
    time["rpa"] = round(time_rpa,3)
    time["eu"] = round(time_eu,3)
    time["fotda_s"] = round(time_fs,3)
    time["fotda_l1l2"] = round(time_fg,3)
    time["botda_s"] = round(time_bs,3)
    time["botda_l1l2"] = round(time_bg,3)
    
    print('ACC')
    print(acc)
    print('CT')
    print(time)
    

Running testing RUN=0
ACC
{'sc': 0.55, 'sr': 0.55, 'ea': 0.7, 'fotda_s': 0.75, 'fotda_l1l2': 0.65, 'botda_s': 0.9, 'botda_l1l2': 0.9}
CT
{'sr': 2.256, 'rpa': 3.2, 'eu': 1.278, 'fotda_s': 0.012, 'fotda_l1l2': 0.118, 'botda_s': 0.006, 'botda_l1l2': 0.082}
Running testing RUN=1
ACC
{'sc': 0.7, 'sr': 0.8, 'ea': 0.85, 'fotda_s': 0.8, 'fotda_l1l2': 0.8, 'botda_s': 0.75, 'botda_l1l2': 0.85}
CT
{'sr': 1.224, 'rpa': 2.571, 'eu': 1.577, 'fotda_s': 0.012, 'fotda_l1l2': 0.094, 'botda_s': 0.008, 'botda_l1l2': 0.08}
Running testing RUN=2
ACC
{'sc': 0.65, 'sr': 0.75, 'ea': 0.7, 'fotda_s': 0.75, 'fotda_l1l2': 0.8, 'botda_s': 0.8, 'botda_l1l2': 0.8}
CT
{'sr': 0.949, 'rpa': 2.313, 'eu': 1.266, 'fotda_s': 0.008, 'fotda_l1l2': 0.212, 'botda_s': 0.01, 'botda_l1l2': 0.1}
Running testing RUN=3
ACC
{'sc': 0.75, 'sr': 0.8, 'ea': 0.8, 'fotda_s': 0.8, 'fotda_l1l2': 0.8, 'botda_s': 0.8, 'botda_l1l2': 0.8}
CT
{'sr': 0.897, 'rpa': 2.809, 'eu': 1.343, 'fotda_s': 0.013, 'fotda_l1l2': 0.175, 'botda_s': 0.009, 'botda_l